Sree Ganeshaaya Namaha


# Kaggle Competion - Santander Customer Transaction Prediction
https://www.kaggle.com/c/santander-customer-transaction-prediction

This Kernel got 0.74 in the Kaggle.


In [1]:

from datetime import datetime
import time

import pandas as pd

import numpy as np
import itertools


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from matplotlib import pyplot as plt

from sklearn.metrics import roc_curve, confusion_matrix
from sklearn.decomposition import PCA
from sklearn import preprocessing


import warnings
from sklearn.exceptions import DataConversionWarning
warnings.simplefilter(action='ignore', category=DataConversionWarning)

import lightgbm as lgb


Few common functions

In [2]:
#print the timestamp along with the log message
def myprint(*argv):
    sstr=datetime.now()
    for arg in argv:
        sstr="{} {}".format(sstr,arg)
    print(sstr)

starttime=time.time()


# Read Train Data

In [3]:

df1=pd.read_csv("../input/train.csv")

cols=df1.columns
scols=""
for col in cols:
    scols="{} {}".format(scols,col)
myprint("Columns:")
myprint(scols)

row1=df1.iloc[0]
srow1=""
for col in cols:
    srow1="{}\n{}:{}".format(srow1, col, row1[col])
myprint("First Row:")
myprint(srow1)

arr1=df1.values#convert the dataframe to list
target_arr=arr1[:,1]#second column is target
train_params=arr1[:,2:]

myprint("target_arr=",target_arr.shape)
data_arr=[]
for i,params in enumerate(train_params):
    data_arr.append((target_arr[i],params))

2019-04-16 20:28:40.600583 Columns:
2019-04-16 20:28:40.600839  ID_code target var_0 var_1 var_2 var_3 var_4 var_5 var_6 var_7 var_8 var_9 var_10 var_11 var_12 var_13 var_14 var_15 var_16 var_17 var_18 var_19 var_20 var_21 var_22 var_23 var_24 var_25 var_26 var_27 var_28 var_29 var_30 var_31 var_32 var_33 var_34 var_35 var_36 var_37 var_38 var_39 var_40 var_41 var_42 var_43 var_44 var_45 var_46 var_47 var_48 var_49 var_50 var_51 var_52 var_53 var_54 var_55 var_56 var_57 var_58 var_59 var_60 var_61 var_62 var_63 var_64 var_65 var_66 var_67 var_68 var_69 var_70 var_71 var_72 var_73 var_74 var_75 var_76 var_77 var_78 var_79 var_80 var_81 var_82 var_83 var_84 var_85 var_86 var_87 var_88 var_89 var_90 var_91 var_92 var_93 var_94 var_95 var_96 var_97 var_98 var_99 var_100 var_101 var_102 var_103 var_104 var_105 var_106 var_107 var_108 var_109 var_110 var_111 var_112 var_113 var_114 var_115 var_116 var_117 var_118 var_119 var_120 var_121 var_122 var_123 var_124 var_125 var_126 var_127 var_128

# Read Test Data

In [4]:

    
df_test=pd.read_csv("../input/test.csv")
arrtmp=df_test.values#convert the dataframe to list
test_codes=arrtmp[:,0]#first column is the code.
test_params=arrtmp[:,1:]#
test_arr=[]
for i in range(0,len(test_params)):
    test_arr.append((test_codes[i],test_params[i]))

myprint("train_params=",train_params.shape)
myprint("test_params=",test_params.shape)



2019-04-16 20:29:23.587067 train_params= (200000, 200)
2019-04-16 20:29:23.588033 test_params= (200000, 200)


# Standard Scaler - fit for train and test params

In [5]:
all_params=np.concatenate(((np.array(train_params)),np.array(test_params)), axis=0)
myprint("all_params=",all_params.shape)

scaler=preprocessing.StandardScaler().fit(all_params)

#minmax_scaled_arr=minmax_scaler.transform(param_arr1)
#myprint ("minmax_scaled_arr=",minmax_scaled_arr.shape)

2019-04-16 20:29:27.416240 all_params= (400000, 200)


# PCA - Fit on train data to reduce dimensions

In [7]:
pca_cols=160
pca = PCA(n_components=pca_cols)
pca.fit(scaler.transform(all_params))

train_scaled_params=scaler.transform(train_params)
train_pca_params=pca.transform(train_scaled_params)



# Fit LightGBM on the train data and get feature importances

In [8]:

ds=lgb.Dataset(train_pca_params,target_arr)
config_params = {'num_leaves': 13,
         'min_data_in_leaf': 80,
          'min_sum_hessian_in_leaf': 10.0,
         'objective': 'binary',
          'boost_from_average': False,
         'max_depth': -1,
         'learning_rate': 0.0083,
         'boost': 'gbdt',
         'bagging_freq': 5,
         'tree_learner': "serial",
         'bagging_fraction': 0.333,
         'feature_fraction': 0.041,
         #'reg_alpha': 1.738,
         #'reg_lambda': 4.99,
         'metric': 'auc',
         #'min_gain_to_split': 0.01077313523861969,
         #'min_child_weight': 19.428902804238373,
         'num_threads': 8}
lgb_model=lgb.train(config_params,ds)
feature_importance=lgb_model.feature_importance()
dict_f_imp={}
for i,val in enumerate(feature_importance):
    dict_f_imp[i]=val


# Consider only the features that have importance > 1

In [10]:
sorted_features=sorted(dict_f_imp.items(),key=lambda x: x[1],reverse=True)
features=[k for (k,v) in sorted_features if v>1]
print("features=",features)
num_cols=len(features)
print(num_cols)

features= [0, 71, 116, 34, 89, 2, 147, 72, 125, 138, 56, 33, 43, 95, 5, 22, 94, 155, 81, 82, 129, 144, 7, 18, 44, 67, 84, 92, 96, 120, 151, 13, 17, 30, 70, 86, 87, 101, 104, 110, 113, 140, 142, 149, 154, 32, 41, 42, 47, 61, 63, 69, 75, 83, 99, 102, 105, 127, 153, 59, 64, 93, 106, 111, 118, 121, 126, 132, 134, 156, 157, 158, 6, 9, 19, 21, 28, 35, 50, 51, 58, 77, 80, 98, 108, 117, 139, 145, 148, 1, 4, 15, 25, 27, 37, 85, 109, 115, 122, 159, 8, 10, 24, 29, 45, 48, 60, 73, 78, 97, 124, 128, 131, 135, 150, 152, 11, 36, 38, 53, 62, 65, 68, 90, 137, 141, 146, 14, 39, 40, 49, 55, 79, 100, 103, 107, 114, 23, 31, 88, 91, 136]
142



# Model - contains two LSTM networks 

In [11]:


def get_reduced_data_arr(raw_params_arr):
    out=[]
    for raw_params in raw_params_arr:
        d2=[]
        for indx in features:
            d2.append(raw_params[indx])
        out.append(d2)
    return out


class Model(nn.Module):
    def __init__(self,scaler,pca,num_cols):
        super(Model,self).__init__()
        self.scaler=scaler
        self.pca=pca
        input_size=num_cols
        myprint("input_size=",input_size)
        self.hidden_size=int(num_cols * 0.1)
        self.hidden_size2=int(self.hidden_size * 0.1)
        if ( self.hidden_size2 < 10):
            self.hidden_size2=10
        myprint("hidden_size=",self.hidden_size)
        myprint("hidden_size2=",self.hidden_size2)
        self.layers=1
        
        self.lstm=nn.LSTM(input_size,self.hidden_size,self.layers, bidirectional=True, 
                           dropout=0.1, batch_first=True)
        
        self.lstm2=nn.LSTM(self.hidden_size*2,self.hidden_size2,self.layers, bidirectional=True, 
                           dropout=0.1, batch_first=True)
        
        self.op_classes=2
        self.fc=nn.Linear(self.hidden_size2*2,self.op_classes)

        self.figcnt=0
        
    def forward(self,indata,h=None,c=None,h2=None,c2=None):
        #apply scaler and pca
        indata=self.pca.transform(self.scaler.transform(indata))
        indata=get_reduced_data_arr(indata)
        indata=torch.Tensor(indata)
        indata=indata.unsqueeze(1)
        
        if h is None:
            h = torch.zeros(self.layers*2, indata.shape[0], self.hidden_size)
            c = torch.zeros(self.layers*2, indata.shape[0], self.hidden_size)
            
        _, (h,c)=self.lstm(indata,(h,c))
        cat=torch.cat((h[-2,:,:],h[-1,:,:]),dim=1)        
        cat=cat.unsqueeze(1)

        if h2 is None:
            h2 = torch.zeros(self.layers*2, cat.shape[0], self.hidden_size2)
            c2 = torch.zeros(self.layers*2, cat.shape[0], self.hidden_size2)
        
        _, (h2,c2)=self.lstm2(cat,(h2,c2))        
        cat2=torch.cat((h2[-2,:,:],h2[-1,:,:]),dim=1)        
            
        out = self.fc(cat2)
        return out, (h,c), (h2,c2)    
        
    def round_prediction(self,preds_sigmoid):    
        #round predictions to the closest integer
        #preds=np.array([0 if i<0.7 else 1 for i in preds_sigmoid])
        preds=np.array([0 if i>j else 1 for [i,j] in preds_sigmoid])
        return preds
        
    def binary_accuracy(self,rounded_preds, y):
        """
        Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
        """
        #round predictions to the closest integer
        
        correct = (torch.Tensor(rounded_preds) == torch.Tensor(y)).float() #convert into float for division     
        acc = correct.sum()/len(correct)
        return acc, correct.sum().item()        
    
    def plot_metrics(self,y_pred_rounded,batch_target):                        

        # Plot non-normalized confusion matrix
        plt.figure(self.figcnt)
        self.figcnt+=1
        self.plot_confusion(batch_target,y_pred_rounded) 

        #fpr, tpr, thresholds = roc_curve(batch_target, y_pred_rounded, pos_label=2)
        #myprint("fpr=",fpr,", tpr=",tpr,",thresholds=",thresholds)
    
    def plot_confusion(self,
             batch_target,
             pred_rounded,
             classes=[0,1],
             normalize=False,
             title='Confusion matrix',
             cmap=plt.cm.Blues):

        cm=confusion_matrix(batch_target, pred_rounded)#, labels=class_names)
        myprint(cm)

        """
        This function prints and plots the confusion matrix.
        Normalization can be applied by setting `normalize=True`.
        """
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            myprint("Normalized confusion matrix")
        else:
            myprint('Confusion matrix, without normalization')
    
        
    
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)
    
        fmt = '.2f' if normalize else 'd'
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.tight_layout()
        plt.show()
        
    def plot_prediction(self,y_pred_sigmoid,y_pred_rounded):
        y=[]
        cnt=1
        for i in y_pred_sigmoid:
            y.append(cnt)
            cnt=cnt+1 
    
        plt.figure(self.figcnt)
        plt.title("sigmoid_{}".format(self.figcnt))
        #plt.scatter(y,y_pred_sigmoid,s=5)
        x=[i for [i,j] in y_pred_sigmoid]
        y=[j for [i,j] in y_pred_sigmoid]
        #print(x)
        #print(y)
        plt.scatter(x,y,s=5)
        plt.show()
        self.figcnt+=1
                        
        #plt.figure(self.figcnt)
        #plt.title("rounded_{}".format(self.figcnt))
        #plt.scatter(y,y_pred_rounded,s=5)
        #plt.show()
        #self.figcnt+=1
    


# Upsampling the data - as train data has very less number of samples with class=1 

In [12]:
class_0=[(cls,params) for (cls,params) in data_arr if cls==0]
class_1=[(cls,params) for (cls,params) in data_arr if cls==1]
total=len(target_arr)
num_0=len(class_0)
num_1=len(class_1)
myprint("total=",total," class0=",num_0, num_0/total, " class1=",num_1, num_1/total)


# Recreate Training set.
diff=num_0-num_1
myprint("diff=",diff)

j=0
for i in range (0,diff):    
    if  i % 50000 == 0:
        myprint (i)
    pos=np.random.randint(0,len(data_arr))
    
    #insert class1 sample randomly into train_arr.
    data_arr.insert(pos,class_1[j])
    
    j+=1
    if j >= len(class_1):
        j=0
myprint("done")

new_class_0=[(cls,params) for (cls,params) in data_arr if cls==0]
new_class_1=[(cls,params) for (cls,params) in data_arr if cls==1]
new_total=len(data_arr)
new_num_0=len(new_class_0)
new_num_1=len(new_class_1)
myprint("total=",new_total," new_class0=",new_num_0, new_num_0/new_total, " new_class1=",new_num_1, new_num_1/new_total)


2019-04-16 20:32:09.649245 total= 200000  class0= 179902 0.89951  class1= 20098 0.10049
2019-04-16 20:32:09.649245 diff= 159804
2019-04-16 20:32:09.649245 0
2019-04-16 20:32:14.812816 50000
2019-04-16 20:32:20.980441 100000
2019-04-16 20:32:28.528370 150000
2019-04-16 20:32:30.177696 done
2019-04-16 20:32:30.354297 total= 359804  new_class0= 179902 0.5  new_class1= 179902 0.5


# Create Optimizer, Loss and Model

In [18]:
model=Model(scaler,pca,num_cols)
#optimizer=optim.SGD(model.parameters(),lr=0.01,momentum=0.9)

lr=0.1

optimizer=optim.Adam(model.parameters(), lr=lr)

#decay LR by a factor of 0.1 every 2 epochs
step_size=2
scheduler=lr_scheduler.StepLR(optimizer,step_size=step_size,gamma=0.1)

#loss_fn=torch.nn.MSELoss(reduction="sum")
#loss_fn=nn.CrossEntropyLoss()#we don't use this here as it is btter suited for multi class classificaiton.

#BCEWithLogitsLoss combines the sigmoid layer and the BCELoss in one single class but is numerically more stable and hence, should be preferred.
#Note here that you don’t need to pass the input tensor to the sigmoid layer before training with BCEWithLogitsLoss.
loss_fn=nn.BCEWithLogitsLoss()#binary cross entropy loss
h,c,h2,c2=None,None,None,None

batch_size=10000
scheduler.step()        


2019-04-16 20:35:30.220003 input_size= 142
2019-04-16 20:35:30.220003 hidden_size= 14
2019-04-16 20:35:30.220003 hidden_size2= 10


c:\myprograms\python\python35\lib\site-packages\torch\nn\modules\rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


# Train function - Shuffle the data for every epoch.

In [19]:
def train(epochs,h,c,h2,c2):    
    
    model.train()
    loss_num=1
    for epoch in range(0,epochs):
        
        if scheduler.get_lr()[0] > 0.0001 and loss_num<0.5:
            scheduler.step()        
        
        for i in range (0,10):                    
            np.random.shuffle(data_arr)
        
        
        epoch_loss=0
        batch=0
        total=0
        correct=0
        cnt=0
        
        lth=len(data_arr)
        for b_start in range(0,lth, batch_size):
            b_end=b_start+batch_size
            batch+=1
            cnt=cnt+1
            #handle length of last batch
            if b_end > lth:
                b_end=lth
                b_start=lth-batch_size
            b_data=data_arr[b_start:b_end]
            b_params=[params for (cls,params) in b_data]
            
            
            #break
            b_target=[cls for (cls,params) in b_data]
            b_target_2d=[[1,0] if cls==0 else [0,1] for cls in b_target]
            b_target_2d=torch.Tensor(b_target_2d)

            optimizer.zero_grad()
            
            pred,(h,c),(h2,c2)=model(b_params,h,c,h2,c2)
            
            loss=loss_fn(pred,b_target_2d)
            loss_num=loss.item()  

            #loss.backward(retain_graph=True)
            loss.backward()
            optimizer.step()

            ## detach the gru from the previous history/graph.
            ## Otherwise, it will hog the memory.
            h.detach_()
            c.detach_()
            h2.detach_()
            c2.detach_()
            

            
            pred_sigmoid=torch.sigmoid(pred)
            #print (pred_sigmoid)
            pred_round=model.round_prediction(pred_sigmoid.detach().numpy())
            #print(pred_round)
            #print(b_target)
            acc,correct=model.binary_accuracy(pred_round,b_target)

            if cnt % 1 == 0:                
                myprint("acc=",acc,"correct predictions=",correct, "loss=",loss_num, "epoch=",epoch, "batch=",batch,"lr=", scheduler.get_lr())
                #model.plot_metrics(pred_round,b_target)
            nowtime=time.time()
            if (nowtime-starttime) >= 330*60:
                print("TIMEOUT")
                print("acc=",acc,"correct predictions=",correct, "loss=",loss_num, "epoch=",epoch, "batch=",batch,"lr=", scheduler.get_lr())

                return (h,c), (h2,c2)
                
    myprint("acc=",acc,"correct predictions=",correct, "loss=",loss_num, "epoch=",epoch, "batch=",batch,"lr=", scheduler.get_lr())
    return (h,c), (h2,c2)
        


In [20]:
(h,c),(h2,c2)=train(140,h,c,h2,c2)



2019-04-16 20:35:38.103260 acc= 0.5080000162124634 correct predictions= 5080.0 loss= 0.6987212896347046 epoch= 0 batch= 1 lr= [0.1]
2019-04-16 20:35:40.570763 acc= 0.5692999958992004 correct predictions= 5693.0 loss= 0.6858659982681274 epoch= 0 batch= 2 lr= [0.1]
2019-04-16 20:35:42.987218 acc= 0.614300012588501 correct predictions= 6143.0 loss= 0.6640947461128235 epoch= 0 batch= 3 lr= [0.1]
2019-04-16 20:35:45.562483 acc= 0.6543999910354614 correct predictions= 6544.0 loss= 0.6286553740501404 epoch= 0 batch= 4 lr= [0.1]
2019-04-16 20:35:48.214657 acc= 0.6970999836921692 correct predictions= 6971.0 loss= 0.5826341509819031 epoch= 0 batch= 5 lr= [0.1]
2019-04-16 20:35:50.610032 acc= 0.7141000032424927 correct predictions= 7141.0 loss= 0.5667421817779541 epoch= 0 batch= 6 lr= [0.1]
2019-04-16 20:35:53.139798 acc= 0.7190999984741211 correct predictions= 7191.0 loss= 0.5759921669960022 epoch= 0 batch= 7 lr= [0.1]
2019-04-16 20:35:55.784837 acc= 0.7419000267982483 correct predictions= 7419.

KeyboardInterrupt: 

# Test function to write the submission.csv file

In [21]:
def test_fn(): 
    myprint("TEST STARTED")
    
    ffile=open("submission.csv","w")
    ffile.write("ID_code,target\n")
    
    #set the mode to eval.
    model.eval()
    
    lth=len(test_arr)
    
    for i in range (0,lth,batch_size):
        end=i+batch_size        
        
        if end > lth:
            end=lth
            
        batch_input=test_params[i:end]                    
        batch_id=test_codes[i:end]
                            
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            pred,(_,_),(_,_)=model(batch_input)            
        
            pred_sigmoid=torch.sigmoid(pred)
        
            pred_round=model.round_prediction(pred_sigmoid.detach().numpy())            
            
            
            for cnt,iid in enumerate(batch_id):
                lline="{},{}\n".format(iid,pred_round[cnt])            
                ffile.write(lline)                            

    ffile.close()
    
    print("DONE")   
     





In [22]:
test_fn()

2019-04-16 20:37:03.666013 TEST STARTED
DONE
